In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import sys
from pathlib import Path
# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

In [3]:
from langgraph.graph import add_messages
from langchain.tools import tool
from langchain.messages import ToolMessage, HumanMessage, AnyMessage
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition
from scripts.retrieve import load_retriever
from utils.utils import format_context
from reranker.rrf import ReciprocalRankFusion
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_core.documents import Document
from config import output_path_prefix
import pickle

with open(f"{output_path_prefix}_split_documents.pkl", "rb") as f:
        split_documents = pickle.load(f)

@tool
def retriever(query: str) -> list[Document]:
    """Retrieve documents from the vector database.

    Args:
        query: The query to retrieve documents from the vector database.
    """
    embeddings = UpstageEmbeddings(model="embedding-passage")
    bm25_retriever, faiss_retriever = load_retriever(split_documents, embeddings, kiwi=False, search_k=10)
    retrieved_docs_faiss = faiss_retriever.invoke(query)
    retrieved_docs_bm25 = bm25_retriever.invoke(query)
    retrieved_docs_faiss = ReciprocalRankFusion.calculate_rank_score(retrieved_docs_faiss)
    retrieved_docs_bm25 = ReciprocalRankFusion.calculate_rank_score(retrieved_docs_bm25)
    retrieved_docs = retrieved_docs_faiss + retrieved_docs_bm25
    rrf_docs = ReciprocalRankFusion.get_rrf_docs(retrieved_docs, cutoff=4)
    context = format_context(rrf_docs)

    return {"documents": rrf_docs, "context": context}


tools = [retriever]
tools_by_name = {tool.name: tool for tool in tools}

def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result, "documents": observation["documents"], "context": observation["context"]}

In [5]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal

In [6]:
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

In [73]:
class Route(BaseModel):
    step: Literal["vector", "web_search"] = Field(
        ..., description="Given a user question choose to route it to web search or a vectorstore."
    )

router = llm.with_structured_output(Route)

In [49]:
ROUTE_PROMPT = """
You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search.
question: {question}
"""

In [50]:
question = "What are the types of agent memory?"


In [51]:
prompt = ROUTE_PROMPT.format(question=question)
router.invoke([{"role": "user", "content": prompt}])

Route(step='vector')

In [16]:
class GradeDocuments(BaseModel):  
    """Grade documents using a binary score for relevance check."""

    binary_score: str = Field(
        description="Relevance score: 'yes' if relevant, or 'no' if not relevant"
    )

GRADE_PROMPT = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n "
    "Here is the retrieved document: \n\n {context} \n\n"
    "Here is the user question: {question} \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
)

grader = llm.with_structured_output(GradeDocuments)

In [17]:
question = "AI Index 2025 연례보고서의 발행 기관과 발행 시기는 언제인가?"
context = retriever.invoke(question)["context"]
prompt = GRADE_PROMPT.format(context=context, question=question)

/home/jake/RAG-end-to-end/faiss_index


In [18]:
retriever.invoke(question)

/home/jake/RAG-end-to-end/faiss_index


{'documents': [Document(id='7bc1c690-e8d4-40e0-a181-e3bb52bf7166', metadata={'page': 3, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 2, 'rank_score': 0.06060606060606061}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\nC O N TEN T\n===========\n\nⅠ. AI Index 2025 주요내용 P.1  \n1. 연구개발 P.1  \n2. 기술 성능 P.4  \n3. 책임 있는 AI P.7  \n4. 경제 P.9  \n5. 과학 및 의료 P.11  \n6. 정책 및 거버넌스 P.14  \n7. 교육 P.16  \n8. 여론 P.18  \nⅡ. 시사점 P.20  \n참고문헌 P.23'),
  Document(id='7dda74c8-8555-4f08-84a2-6e67e44e34ab', metadata={'page': 1, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 0, 'rank_score': 0.05928509154315606}, page_content='ISSUE REPORT l 2025.04.15. IS-200\n\n\nA I Index 2025 주요 내용과 시사점\n=========================\n\nSummary and Implications of 2025 AI Index Report\n\n안성원/임영모/유재흥/안미소/장진철/이해수/김지원/임정주'),
  Document(id='07a53bfd-4d9c-4d59-865b-314480cd6981', metadata={'page': 28, 'image_id': [], 'image_path': [], 'tex

In [18]:
grader.invoke([{"role": "user", "content": prompt}])

GradeDocuments(binary_score='yes')

In [ ]:
GENERATE_PROMPT = """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context}
"""

In [21]:
question = "AI Index 2025 연례보고서의 발행 기관과 발행 시기는 언제인가?"
context = retriever.invoke(question)["context"]

/home/jake/RAG-end2end/faiss_index


In [22]:
prompt = GENERATE_PROMPT.format(question=question, context=context)
answer = llm.invoke([{"role": "user", "content": prompt}])
answer

AIMessage(content='발행 기관: AI Index 운영위원회(AI Index Steering Committee) 및 스탠포드대학교 인간중심 AI 연구소(Institute for Human‑Centered AI).  \n발행 시기: 2025년 4월.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 251, 'prompt_tokens': 1254, 'total_tokens': 1505, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cw3Chx9XbTGpkcQ4uV6F7FtuEbOoZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--6774133c-07e8-4fde-aef5-37e209512304-0', usage_metadata={'input_tokens': 1254, 'output_tokens': 251, 'total_tokens': 1505, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

In [52]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )
hallucination_grader = llm.with_structured_output(GradeHallucinations)

In [24]:
HALLUCINATION_PROMPT = """
You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts.
     Here is the LLM generation: \n\n {answer} \n\n"
     Here is the set of facts: \n\n {context} \n\n"
"""

prompt = HALLUCINATION_PROMPT.format(answer=answer, context=context)
res = hallucination_grader.invoke([{"role": "user", "content": prompt}])
res

GradeHallucinations(binary_score='yes')

In [25]:
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

answer_grader = llm.with_structured_output(GradeAnswer)


In [26]:
ANSWER_PROMPT = """
You are a grader assessing whether an answer addresses / resolves a question \n 
Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question.
Here is the question: {question} \n
Here is the answer: {answer} \n
"""

prompt = ANSWER_PROMPT.format(question=question, answer=answer.content)
res = answer_grader.invoke([{"role": "user", "content": prompt}])
res



GradeAnswer(binary_score='yes')

In [27]:
REWRITE_PROMPT = """
You a question re-writer that converts an input question to a better version that is optimized \n 
for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning.
Here is the initial question: \n\n {question} \n Formulate an improved question.
"""
question = "AI Index 2025 연례보고서의 발행 기관과 발행 시기는 언제인가?"
prompt = REWRITE_PROMPT.format(question=question)
llm.invoke([{"role": "user", "content": prompt}])


AIMessage(content='AI Index 2025 연례보고서(AI Index Report 2025 Annual Report)의 발행 주체(발행 기관)는 어디이며, 공식 발행일(출간일·배포일)은 언제인가?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 440, 'prompt_tokens': 82, 'total_tokens': 522, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cw3KMKvA7QZamrpkduQjuQyuLN9cV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2e95395a-c88d-41d8-afd5-c540e0d6c16c-0', usage_metadata={'input_tokens': 82, 'output_tokens': 440, 'total_tokens': 522, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}})

In [7]:
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(max_results=3)

/home/jake/RAG-end-to-end/.venv/lib/python3.13/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/home/jake/RAG-end-to-end/.venv/lib/python3.13/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


In [15]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[Document]
    context: str
    step: str

In [21]:
from langchain_core.documents import Document

def retrieve(state: GraphState) -> GraphState:
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    result = retriever.invoke(question)
    return {"context": result["context"], "documents": result["documents"], "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    context = state["context"]
    # RAG generation
    GENERATE_PROMPT = """
        You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 
        Use three sentences maximum and keep the answer concise.
        Question: {question} 
        Context: {context}
    """
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    generation = llm.invoke([{"role": "user", "content": prompt}])
    return {"context": context, "question": question, "generation": generation}

class GradeDocuments(BaseModel):  
    """Grade documents using a binary score for relevance check."""

    binary_score: str = Field(
        description="Relevance score: 'yes' if relevant, or 'no' if not relevant"
    )

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    GRADE_PROMPT = (
        "You are a grader assessing relevance of a retrieved document to a user question. \n "
        "Here is the retrieved document: \n\n {document} \n\n"
        "Here is the user question: {question} \n"
        "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
        "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
    )

    retrieval_grader = llm.with_structured_output(GradeDocuments)
    
    # Score each doc
    filtered_docs = []
    for d in documents:
        prompt = GRADE_PROMPT.format(question=question, document=d.page_content)
        score = retrieval_grader.invoke([{"role": "user", "content": prompt}])
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]
    REWRITE_PROMPT = """
        You a question re-writer that converts an input question to a better version that is optimized \n 
        for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning.
        Here is the initial question: \n\n {question} \n Formulate an improved question.
    """
    question = "AI Index 2025 연례보고서의 발행 기관과 발행 시기는 언제인가?"
    prompt = REWRITE_PROMPT.format(question=question)
    better_question = llm.invoke([{"role": "user", "content": prompt}])
    return {"documents": documents, "question": better_question}


def run_web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})['results']
    context = "\n".join([d["content"] for d in docs])
    web_results = [Document(page_content=d["content"]) for d in docs]

    return {"documents": web_results, "context": context, "question": question}

In [22]:
class Route(BaseModel):
    step: Literal["vectorstore", "web_search"] = Field(
        ..., description="Given a user question choose to route it to web search or a vectorstore."
    )

router = llm.with_structured_output(Route)
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
   
    ROUTE_PROMPT = """
        You are an expert at routing a user question to a vectorstore or web search.
        The vectorstore contains documents related to 
        The following context is a summary report published by the Software Policy & Research Institute (SPRi). 
        It discusses the findings of the original 'AI Index 2025' published by Stanford University.
        Use the vectorstore for questions on these topics. Otherwise, use web-search.
        question: {question}
    """
    question = state["question"]
    prompt = ROUTE_PROMPT.format(question=question)
    source = router.invoke([{"role": "user", "content": prompt}])
    if source.step == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "run_web_search"
    elif source.step == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    context = state["context"]
    generation = state["generation"]
    
    hallucination_grader = llm.with_structured_output(GradeHallucinations)
    HALLUCINATION_PROMPT = """
        You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
        Give a binary score 'yes' or 'no'. 'Yes' means that the generation is grounded in / supported by the set of facts.
        Here is the LLM generation: \n\n {generation} \n\n"
        Here is the set of facts: \n\n {context} \n\n"
    """

    prompt = HALLUCINATION_PROMPT.format(generation=generation, context=context)
    score = hallucination_grader.invoke([{"role": "user", "content": prompt}])
    grade = score.binary_score

    ANSWER_PROMPT = """
        You are a grader assessing whether an generation addresses / resolves a question \n 
        Give a binary score 'yes' or 'no'. Yes' means that the generation addresses the question.
        Here is the question: {question} \n
        Here is the generation: {generation} \n
    """

    prompt = ANSWER_PROMPT.format(question=question, generation=generation)
    answer_grader = llm.with_structured_output(GradeAnswer)
    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke([{"role": "user", "content": prompt}])
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [23]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("run_web_search", run_web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "run_web_search": "run_web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("run_web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [19]:
from pprint import pprint

# Run
inputs = {
    "question": "What player at the Bears expected to draft first in the 2024 NFL draft?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
"Node 'run_web_search':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
AIMessage(content='They expected to draft USC quarterback Caleb Williams with the No. 1 overall pick.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 181, 'total_tokens': 335, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CwA6YHicIDFdm4f6OB27ct8v6j5B0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba3b1-

In [24]:
# Run
inputs = {"question": "SPRI 2025 보고서에서 설명한 AI 트렌드는 무엇인가?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
/home/jake/RAG-end-to-end/faiss_index
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
AIMessage(content='SPRi 보고서는 주요 트렌드로 연구개발 경쟁 심화(미국 대비 중국 약진), AI 성능의 급격한 향상과 새로운 벤치마크 등장 및 고성능 모델 간 성능 격차 축소를 꼽았습니다. 또한 AI 활용이 과학·의료 분야에서 확산되며 책임 있는 AI 노력과 각국의 규제·법률 증가, 2024년 글로벌 투자 회복과 증가를 강조합니다. 마지막으로 AI·CS 교육의 빠른 확산으로 전문가 배출이 가속화되는 한편, 여론은 낙관적이지만 공정성에 대한 신뢰는 감소하고 있다고 설명합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 477, 'prompt_tokens': 1183, 'tota

In [5]:
import pandas as pd

In [6]:
df_correct = pd.read_csv("../outputs/SPRI_2025_output_eval_correct.csv")

In [7]:
df_correct.head(1)

,Unnamed: 0,query,answer,page_number,outputs.answer,outputs.page_number,retrieved_contexts,correctness,explanation
0,0,"세계의 CS 교육 보급 수준(2024), 미국 고등학교 CS 과정의 비율 변화(20...",2024년 전 세계 국가의 약 2/3가 초중고 차원에서 컴퓨터과학(CS) 과정을 도...,21,"먼저 출처 구분(명확히)\n- 요약(본 문서)의 저자: ""SPRi 이슈리포트 IS-...","[21, 22, 22, 27]",### Context #1<document><page_content>SPRi 이슈리...,True,Step 1 — World CS adoption (2024): The student...


In [8]:
df_incorrect = df_correct[df_correct["correctness"] == False].copy()

for i, row in df_incorrect.iterrows():
    print("index : ", i)
    print("page_numbers : ", row["page_number"])
    print("outputs.page_number : ", row["outputs.page_number"])
    print("query : ", row["query"])
    print("answer : ", row["answer"])
    print("outputs.answer : ", row["outputs.answer"])
    print("correctness : ", row["correctness"])
    print("explanation : ", row["explanation"])
    print("-"*100)

index :  4
page_numbers :  9
outputs.page_number :  [9, 10, 7, 11]
query :  AI 성능이 급격히 향상되었다고 했는데, 신규 벤치마크 점수는 1년 만에 최대 몇 퍼센트까지 향상되었고, 상위권과 10위권 모델의 ELO 점수차 및 상위 2개 모델의 점수차는 각각 얼마인가요?
answer :  신규 벤치마크 점수는 불과 1년 만에 최대 71.7%까지 향상되었고, 상위권과 10위권 모델의 ELO 점수차는 11.9%(`24년)에서 5.4%(`23년)로 감소했으며, 상위 2개 모델의 점수차는 0.7%에 불과합니다.
outputs.answer :  요약 작성자와 원저자 구분
- 요약: "SPRi 이슈리포트 IS-200" (요약문서 작성자: SPRi)  
- 원보고서 출처(원저자): "출처는 Stanford HAI, AI Index Report 2025."

질문 답변(모두 원문 인용 포함)
- 신규 벤치마크 점수 1년 개선치: "불과 1년 만에 벤치마크 점수가 최대 71.7% 까지 향상됨"  
- 모델 간 ELO 점수차(원문): "챗봇 아레나 순위표에서 1위와 10위 모델 간 Elo 점수 차는 기존 11.9%(`23년)에서 5.4%(`25년 초)로 좁혀졌으며, 상위 두 모델 간 격차 또한 4.9%(`23년)에서 0.7%(`25년 말)로 줄어듦"

즉, 원문에 따르면 최대 향상치는 71.7%이며(인용 위 문구 참조), 챗봇 아레나 기준으로 1위 vs 10위 격차는 최근 치(문서 기준) 5.4%, 상위 2개 모델 격차는 최근 0.7%로 보고되어 있습니다(위 인용문 참고).
correctness :  False
explanation :  Step 1 — Benchmark improvement: The student correctly reports the numerical improvement of up to 71.7% in one year, which matches the ground truth.

Step 2 — ELO

In [9]:
len(df_incorrect)

8

In [13]:
df_incorrect.iloc[1]

Unnamed: 0                                                            24
query                  이 문서의 보고서 번호(IS-200), 발행일(2025.04.15), 한국어 제목과...
answer                 보고서 번호: IS-200\n발행일: 2025.04.15\n한국어 제목: A I I...
page_number                                                            1
outputs.answer         먼저 문서에서 확인된 항목(직접 인용표시한 내용)입니다.\n\n- 보고서 번호: "...
outputs.page_number                                      [28, 4, 29, 25]
retrieved_contexts     ### Context #1<document><page_content>SPRi 이슈리...
correctness                                                        False
explanation            채점 기준에 따라 항목별로 비교·검토한 결과 다음과 같습니다.\n\n1) 보고서 번...
Name: 24, dtype: object

In [11]:
embeddings = UpstageEmbeddings(model="embedding-passage")
query = df_incorrect.iloc[1]["query"]
bm25_retriever, faiss_retriever = load_retriever(split_documents, embeddings, kiwi=False, search_k=10)
retrieved_docs_faiss = faiss_retriever.invoke(query)
retrieved_docs_bm25 = bm25_retriever.invoke(query)

/home/jake/RAG-end-to-end/faiss_index


In [26]:
retrieved_docs_faiss = ReciprocalRankFusion.calculate_rank_score(retrieved_docs_faiss)
retrieved_docs_bm25 = ReciprocalRankFusion.calculate_rank_score(retrieved_docs_bm25)
retrieved_docs = retrieved_docs_faiss + retrieved_docs_bm25
rrf_docs = ReciprocalRankFusion.get_rrf_docs(retrieved_docs, cutoff=10)

In [22]:
retrieved_docs_faiss

[Document(id='7dda74c8-8555-4f08-84a2-6e67e44e34ab', metadata={'page': 1, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 0, 'rank_score': 0.03225806451612903}, page_content='ISSUE REPORT l 2025.04.15. IS-200\n\n\nA I Index 2025 주요 내용과 시사점\n=========================\n\nSummary and Implications of 2025 AI Index Report\n\n안성원/임영모/유재흥/안미소/장진철/이해수/김지원/임정주'),
 Document(id='89e4f3ca-f247-4d7a-b9cb-92bb858f6b71', metadata={'page': 28, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 46, 'rank_score': 0.03125}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\n참고문헌\n====\n\n국외문헌\n===='),
 Document(id='7bc1c690-e8d4-40e0-a181-e3bb52bf7166', metadata={'page': 3, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 2, 'rank_score': 0.030303030303030304}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\nC O N TEN T\n===========\n\nⅠ. AI Index 2025 주요내용 P.1  \n1. 연

In [23]:
retrieved_docs

[Document(id='7dda74c8-8555-4f08-84a2-6e67e44e34ab', metadata={'page': 1, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 0, 'rank_score': 0.03225806451612903}, page_content='ISSUE REPORT l 2025.04.15. IS-200\n\n\nA I Index 2025 주요 내용과 시사점\n=========================\n\nSummary and Implications of 2025 AI Index Report\n\n안성원/임영모/유재흥/안미소/장진철/이해수/김지원/임정주'),
 Document(id='89e4f3ca-f247-4d7a-b9cb-92bb858f6b71', metadata={'page': 28, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 46, 'rank_score': 0.03125}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\n참고문헌\n====\n\n국외문헌\n===='),
 Document(id='7bc1c690-e8d4-40e0-a181-e3bb52bf7166', metadata={'page': 3, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 2, 'rank_score': 0.030303030303030304}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\nC O N TEN T\n===========\n\nⅠ. AI Index 2025 주요내용 P.1  \n1. 연

In [27]:
for doc in rrf_docs:
    print(doc.metadata["rank_score"])
    print("-"*100)

0.059027777777777776
----------------------------------------------------------------------------------------------------
0.05798319327731093
----------------------------------------------------------------------------------------------------
0.05789909015715467
----------------------------------------------------------------------------------------------------
0.05131578947368421
----------------------------------------------------------------------------------------------------
0.03225806451612903
----------------------------------------------------------------------------------------------------
0.03125
----------------------------------------------------------------------------------------------------
0.030303030303030304
----------------------------------------------------------------------------------------------------
0.030303030303030304
----------------------------------------------------------------------------------------------------
0.029411764705882353
--------------------

In [28]:
rrf_docs

[Document(id='89e4f3ca-f247-4d7a-b9cb-92bb858f6b71', metadata={'page': 28, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 46, 'rank_score': 0.059027777777777776}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\n참고문헌\n====\n\n국외문헌\n===='),
 Document(id='17749dae-ae82-44b7-a43e-50c0b91faca6', metadata={'page': 4, 'image_id': [], 'image_path': [], 'text_summary': '요약: SPRi 이슈리포트 IS-200은 Stanford HAI의 AI Index 2025를 분석해 현재 글로벌 AI 현황의 핵심 내용을 제시하고, SPRi의 전략적 대응을 제안한다.\n\n- AI 연구개발 경쟁이 매년 치열해지고 있으며, 미국에 대한 중국의 약진이 두드러진다.\n- AI 성능이 매년 빠르게 증가하고 새로운 벤치마크가 등장하며, 고성능 모델 간의 성능 격차가 좁아져 상향 평준화가 진행된다.\n- AI 활용 확산이 과학과 의료의 발전을 이끌고 있으며, 책임 있는 AI를 위한 다양한 노력도 함께 추진되고 있다.\n- 각국은 AI 규제 법률을 늘리는 추세이다.\n- 2024년 글로벌 AI 투자 수준은 경기침체 속 22–23년의 감소에서 크게 회복·증가했다.\n- AI와 컴퓨터과학(CS) 교육이 빠르게 확산되며 AI 전문가의 배출이 가속화되고 있다.\n- AI에 대한 여론은 낙관적 전망이 증가하는 반면, 공정성에 대한 신뢰도는 감소하는 양상을 보인다.', 'image_summary': [], 'id': 3, 'rank_score': 0.05798319327731093}, page_content="SPRi 이슈리포

In [14]:
retrieved_docs_faiss

[Document(id='7dda74c8-8555-4f08-84a2-6e67e44e34ab', metadata={'page': 1, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 0}, page_content='ISSUE REPORT l 2025.04.15. IS-200\n\n\nA I Index 2025 주요 내용과 시사점\n=========================\n\nSummary and Implications of 2025 AI Index Report\n\n안성원/임영모/유재흥/안미소/장진철/이해수/김지원/임정주'),
 Document(id='89e4f3ca-f247-4d7a-b9cb-92bb858f6b71', metadata={'page': 28, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 46}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\n참고문헌\n====\n\n국외문헌\n===='),
 Document(id='7bc1c690-e8d4-40e0-a181-e3bb52bf7166', metadata={'page': 3, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 2}, page_content='SPRi 이슈리포트 IS-200\n\n\n  \nAI Index 2025의 주요 내용 및 시사점\n\nC O N TEN T\n===========\n\nⅠ. AI Index 2025 주요내용 P.1  \n1. 연구개발 P.1  \n2. 기술 성능 P.4  \n3. 책임 있는 AI P.7  \n4. 경제 P.9  \n5. 과학 및 의료 P.11  \n6. 정책 및 거버넌스 P.1

In [12]:
retrieved_docs_bm25

[Document(metadata={'page': 29, 'image_id': [351], 'image_path': ['/images/SPRI_2025_cropped_figure_351.png'], 'text_summary': [], 'image_summary': '다음과 같이 보이는 흑백 배너 이미지입니다.\n\n- 왼쪽에 큰 “OPEN” 로고가 배치되어 있습니다. O 글자 모양이 돋보이며, 나머지 글자도 굵은 서체로 표시됩니다.\n- 로고 오른쪽 쪽에 가로로 배열된 네 개의 작은 사각 아이콘이 있습니다. 각 아이콘은 간단한 도형이나 실루엣으로 구성된 듯 보이나 자세한 내용은 식별하기 어렵습니다.\n- 하단이나 로고 아래쪽에 짧은 한국어 텍스트가 몇 줄 보이지만 글자는 작아 읽기 어렵습니다.\n- 전반적으로 오픈 라이선스/오픈 콘텐츠를 시사하는 디자인 요소의 흑백 배너입니다.', 'id': 50}, page_content='주 의\n===\n\n\n이 보고서는 소프트웨어정책연구소에서 수행한 연구보고서입니다.  \n이 보고서의 내용을 발표할 때에는 반드시  \n소프트웨어정책연구소에서 수행한 연구결과임을 밝혀야 합니다.\n\n![](/images/SPRI_2025_cropped_figure_351.png)\n\n  \n[소프트웨어정책연구소]에 의해 작성된 [SPRI 보고서]는 공공저작물 자유이용허락 표시기준  \n제4유형(출처표시-상업적이용금지-변경금지)에 따라 이용할 수 있습니다.'),
 Document(metadata={'page': 2, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 1}, page_content='이 보고서는 ｢과학기술정보통신부 정보통신진흥기금｣에서 지원받아 제작한 것으로  \n과학기술정보통신부의 공식의견과 다를 수 있습니다.\n\n\n이 보고서의 내용은 연구진의 개인 견해이며, 보고서와 관련한 의문 사항 또는 수정·보완할 필요가  \n있는 

In [37]:
# Run
inputs = {"question": df_incorrect.iloc[1]["query"]}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
/home/jake/RAG-end-to-end/faiss_index
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---
"Node 'generate':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
AIMessage(content='보고서 번호는 IS-200이며 한국어 제목은 "AI Index 2025의 주요 내용 및 시사점"으로 문서에 나와 있습니다. 발행일(2025.04.15)과 영어 부제는 문서에 명시되어 있지 않습니다. 개별 저자 명단도 확인되지 않으며 작성 주체는 소프트웨어정책연구소(SPRi)로 표기되어 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completi